In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy as cp
import numpy as np
import pickle
import timeit
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [10]:
class BIOF050_SelfSup:
    
    
    def __init__(self,data):
        self.data = data
  
    
    '''
        
        Inside this Network class, we can define what we want our neural network to look like!
        We want a network that takes some part of the data as input (input_dim)
        and predicts some of the missing features (output_dim)
        
    '''
    
    class Network(nn.Module):
        def __init__(self,input_dim,output_dim,hidden_dimensions):
            super(BIOF050_SelfSup.Network, self).__init__()
    
            self.input_layer = nn.Linear(input_dim,hidden_dimensions)
            self.layer1 = nn.Linear(hidden_dimensions, hidden_dimensions)
            self.layer2 = nn.Linear(hidden_dimensions, hidden_dimensions)
            self.output_layer = nn.Linear(hidden_dimensions,output_dim)
            self.batch_norm = nn.BatchNorm1d(hidden_dimensions)
            self.relu = nn.ReLU()
            self.tanh = nn.Tanh()
     
            
            
        def forward(self,batch): 
      
            batch = self.input_layer(batch)
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            batch = self.layer1(batch)
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            batch = self.layer2(batch)
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            batch = self.output_layer(batch)
            
            return self.tanh(batch)
    
            
        
        
    def train_test(self,n_epochs,output_dim,hidden_dimensions,batch_size,lr):
            
        ## creating the batches using the batchify function
        train_batches = batchify(self.data,batch_size=batch_size)
        
        '''
        Here is where we define our neural network model - the Network class is inside BIOF050, so we have to call
        it accordingly 
        
        We use the length of our first data point to set the length of our input data (they are all the same)
        
        The output_dim is equal to the number of missing data points we want to predict
        '''
        neural_network = BIOF050_SelfSup.Network(len(train_batches[0][0]),output_dim,hidden_dimensions)
        
        
        '''
        Here, we use the torch.optim package to create our stochastic gradient descent function
        
        neural_network.parameters() reads internal information from our NN 
        (don't worry about that - Adam just requires it)
        
        We will use Adam, a powerful optimization function that optimizes learning rates 
        for the neurons
        
        lr is the learning rate
        '''
        optimizer = optim.Adam(neural_network.parameters(), lr=lr)
        
        
        '''
        Here, we are comparing the predicted values with the actual values for the missing data,
        so we use MSE
        '''
        loss_function = nn.MSELoss()
        
                
        '''
        The train function tells the neural network that it is about to be trained and that it 
        will have to calculate the needed information for optimization 
        
        This function should always be called before training
        '''
        neural_network.train()
        
        
        ''' This loop moves through the data once for each epoch'''
        for i in range(n_epochs):
            
            
            total = 0
            
            ''' This loop moves through each batch and feeds into the neural network'''
            for ii in range(len(train_batches)):
                
                ''' 
                Clears previous gradients from the optimizer - the optimizer,
                in this case, does not need to know what happened last time
                '''
                optimizer.zero_grad()
                
                
                batch = train_batches[ii]
  
                inputs = []
                targets = []
                batch_copy = cp.deepcopy(train_batches[ii])
   
                ''' 
                for each vector in our batch, we remove some of the data, we save it 
                as the target, and then we set those values to zero in the original data
                If zero, these will not be considered (zero *weight = 0)
                
                This is where we do our self-supervision - we are predicting missing 
                data using the rest of it!
                
                '''
                for n in range(len(batch_copy)):
                    selection = np.random.choice(list(range(len(batch_copy[n]))),output_dim,replace=False)
                    targets.append(batch_copy[n][selection])
                    batch_copy[n][selection] = 0
         

                
                ''' 
                Puts our batch into the neural network after converting it to a tensor
                
                Pytorch wants numeric data to be floats, so we will convert to a float as well 
                using np.float32
                
                '''
                predictions = neural_network(torch.tensor(np.asarray(batch_copy).astype(np.float32)))
        
            
                
                ''' 
                We put our predicted values into the loss function to calculate the error for this batch
                
                '''
                loss = loss_function(predictions,torch.tensor(np.asarray(targets).astype(np.float32)))
                
                
                total += loss
                
                '''
                loss.backward calculates the partial derivatives that we need to optimize
                '''
                loss.backward()
                
                
                '''
                optimizer step calculates the weight updates so the neural network can update the weights 
                '''
                optimizer.step()
                
            print(total/len(train_batches))
    
           
        return neural_network
        
   
   


''' Utility Function - function to turn the data into batches'''

def batchify(data,batch_size=16):
    
    batches= []



    for n in range(0,len(data),batch_size):
        if n+batch_size < len(data):
            batches.append(data[n:n+batch_size])
            

    if len(data)%batch_size > 0:
        batches.append(data[len(data)-(len(data)%batch_size):len(data)])
        
    return batches



In [11]:
data = MinMaxScaler().fit_transform(pd.read_csv('data.csv').drop(['score'],1).values)


In [12]:
testclass = BIOF050_SelfSup(data)
model = testclass.train_test(n_epochs=10,output_dim=1,hidden_dimensions=100,batch_size=16,
                             lr=0.001)

tensor(0.0044, grad_fn=<DivBackward0>)


KeyboardInterrupt: 